In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# 指定训练集
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 指定测试集
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# 创建数据加载器
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# 指定设备
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")


# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

# 定义损失函数
loss_fn = nn.CrossEntropyLoss()
# 定义优化器
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


# 定义训练函数
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)

        # 计算预测结果和输出结果相乘多少
        loss = loss_fn(pred, y)

        # 反向传播
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


# 定义测试函数
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


# 定义循环次数，每次循环里面，先训练，再测试
epochs = 30
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.306840  [   64/60000]
loss: 2.303446  [ 6464/60000]
loss: 2.278438  [12864/60000]
loss: 2.268619  [19264/60000]
loss: 2.257334  [25664/60000]
loss: 2.225109  [32064/60000]
loss: 2.234313  [38464/60000]
loss: 2.203902  [44864/60000]
loss: 2.197371  [51264/60000]
loss: 2.166493  [57664/60000]
Test Error: 
 Accuracy: 41.5%, Avg loss: 2.165887 

Epoch 2
-------------------------------
loss: 2.177843  [   64/60000]
loss: 2.140420  [ 6464/60000]
loss: 2.112628  [12864/60000]
loss: 2.107665  [19264/60000]
loss: 2.044683  [

In [13]:
!pip install tensorboard

In [16]:
from torch.utils.tensorboard import SummaryWriter
import random


writer = SummaryWriter(log_dir="tensorboard_test")

offset = random.random()/2
epochs = 10
for epoch_i in range(2, epochs):
    acc = 1 - 2 ** -epoch_i - random.random() / epoch_i - offset
    loss = 2 ** -epoch_i - random.random() / epoch_i - offset
    writer.add_scalar('Accuracy/acc', acc, epoch_i)
    writer.add_scalar('Accuracy/loss', loss, epoch_i)

writer.close()
# tensorboard --logdir . 查看模型训练效果图表化数据

In [18]:
# 查看模型的网络结构
writer = SummaryWriter(log_dir="tensorboard_test")
model = NeuralNetwork()
init_img = torch.zeros((1,1,28,28))
writer.add_graph(model, init_img)
writer.close()